### Cyclical Learning Rates

Using Cyclical Learning Rates you can dramatically reduce the number of experiments required to tune and find an optimal learning rate for your model.

there are two problems with basic learning rate schedules:

1-We dont know what the optimal initial learning rate is.

2-Monotonically decreasing our learning rate may lead to our network getting “stuck” in plateaus of the loss landscape.

In [ ]:
!git clone https://github.com/bckenstler/CLR.git

In [13]:
import os

# initialize the list of class label names
CLASSES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# define the minimum learning rate, maximum learning rate, batch size, step size, CLR method, and number of epochs
MIN_LR = 1e-7
MAX_LR = 1e-2
BATCH_SIZE = 64
STEP_SIZE = 8
CLR_METHOD = "triangular"
NUM_EPOCHS = 96

# define the path to the output training history plot and cyclical learning rate plot
TRAINING_PLOT_PATH = os.path.sep.join(["output", "training_plot.png"])
CLR_PLOT_PATH = os.path.sep.join(["output", "clr_plot.png"])


# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from clr_callback import CyclicLR
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.mobilenet import MobileNet


((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")

# apply mean subtraction to the data
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)


aug = ImageDataGenerator(width_shift_range=0.1,
	height_shift_range=0.1, horizontal_flip=True,
	fill_mode="nearest")


model = MobileNet(
    input_shape=(32, 32, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=10,
    classifier_activation='softmax')


opt = SGD(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

clr = CyclicLR(
	mode= CLR_METHOD,
	base_lr= MIN_LR,
	max_lr= MAX_LR,
	step_size=  STEP_SIZE * (trainX.shape[0] //  BATCH_SIZE))

In [14]:
H = model.fit(
  x= aug.flow(trainX, trainY, batch_size= BATCH_SIZE),
  validation_data=(testX, testY),
  steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
  epochs=NUM_EPOCHS,
  callbacks=[clr],
  verbose=1)

Epoch 1/96
781/781 [==============================] - 47s 56ms/step - loss: 2.5051 - accuracy: 0.1130 - val_loss: 2.4062 - val_accuracy: 0.1240
Epoch 2/96
781/781 [==============================] - 34s 44ms/step - loss: 2.2144 - accuracy: 0.1811 - val_loss: 2.2053 - val_accuracy: 0.2049
Epoch 3/96
781/781 [==============================] - 37s 47ms/step - loss: 1.9570 - accuracy: 0.2622 - val_loss: 1.9537 - val_accuracy: 0.2915
Epoch 4/96
781/781 [==============================] - 34s 43ms/step - loss: 1.8106 - accuracy: 0.3169 - val_loss: 1.7444 - val_accuracy: 0.3314
Epoch 5/96
781/781 [==============================] - 34s 44ms/step - loss: 1.6899 - accuracy: 0.3682 - val_loss: 1.6828 - val_accuracy: 0.3813
Epoch 6/96
781/781 [==============================] - 35s 44ms/step - loss: 1.6058 - accuracy: 0.4061 - val_loss: 1.5757 - val_accuracy: 0.4284
Epoch 7/96
781/781 [==============================] - 35s 44ms/step - loss: 1.5308 - accuracy: 0.4364 - val_loss: 1.5371 - val_accuracy:

In [16]:
predictions = model.predict(x=testX, batch_size=BATCH_SIZE)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=CLASSES))

157/157 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

    airplane       0.78      0.81      0.79      1000
  automobile       0.84      0.90      0.87      1000
        bird       0.77      0.62      0.69      1000
         cat       0.61      0.52      0.56      1000
        deer       0.73      0.69      0.71      1000
         dog       0.67      0.64      0.65      1000
        frog       0.69      0.89      0.78      1000
       horse       0.78      0.82      0.80      1000
        ship       0.90      0.84      0.87      1000
       truck       0.82      0.86      0.84      1000

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000



In [22]:
# plot the training loss and accuracy
N = np.arange(0, NUM_EPOCHS)
# plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('training_plot.png')

# plot the learning rate history
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")
plt.savefig("Learning Rate")